In [216]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
import pickle
from datetime import datetime, timedelta

In [217]:
import requests
from bs4 import BeautifulSoup

# Make a request to the website
url = "https://www.nba.com/game/bos-vs-dal-0042300404/play-by-play?latest=0&period=All"
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

plays = soup.find('script', {"id": '__NEXT_DATA__'})
d = json.loads(plays.string)
play_by_play = d['props']['pageProps']['playByPlay']['actions']

In [218]:
def get_game_links(date):
    # date: string formatted as yyyy-mm-dd
    links = []
    url = f"https://www.nba.com/games?date={date}"
    response = requests.get(url)

    # Parse HTML with bs4
    soup = BeautifulSoup(response.content, "html.parser")
    data = soup.find('script', {"id": '__NEXT_DATA__'})
    d = json.loads(data.string) # bigass json stored as python dict 
    modules = d['props']['pageProps']['gameCardFeed']['modules']
    if modules:
        cards = d['props']['pageProps']['gameCardFeed']['modules'][0]['cards']
        for card in cards:
            url_chunk = card['cardData']['actions'][-1]['resourceLocator']['resourceUrl']
            links.append('https://www.nba.com' + url_chunk)
        return links
    else:
        return

In [219]:
def parse_game(ref_foul_data, link):

    # first, check who the officials are and add 1 game
    # to their count in the dictionary
    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")
    plays = soup.find('script', {"id": '__NEXT_DATA__'})
    d = json.loads(plays.string)
    officials = d['props']['pageProps']['game']['officials']

    for official in officials:
        name = official['nameI']
        if name not in ref_foul_data:
            ref_foul_data[name] = {'num_games': 0}
        ref_foul_data[name]['num_games'] += 1
        
    # next, move on to the play-by-play of the game!
    url = link + 'play-by-play?latest=0&period=All'
    response = requests.get(url)

    # Parse HTML with bs4
    soup = BeautifulSoup(response.content, "html.parser")
    plays = soup.find('script', {"id": '__NEXT_DATA__'})
    d = json.loads(plays.string)
    play_by_play = d['props']['pageProps']['playByPlay']['actions']

    pattern = r'\(([^)]+)\)$' # to extract ref names from event descs
    for play in play_by_play:
        
        if play['actionType'] == 'Foul':
            desc = play['description']
            regex_match = re.search(pattern, desc)
            if regex_match:
                ref = regex_match.group(1)
                ref = ref.replace('.', '. ')
                
            foul_type = play['subType']
            
            if foul_type not in ref_foul_data[ref]:
                ref_foul_data[ref][foul_type] = 0
            
            ref_foul_data[ref][foul_type] += 1
    
    return play_by_play

In [220]:
season_start = datetime(2023, 10, 24).date()
present_day = datetime.now().date()
delta = timedelta(days=1)
scrape_date = season_start


ref_foul_data = {}
# MAIN SCRAPING LOOP
while scrape_date <= present_day:
    print('scraping', scrape_date)
    date_str = scrape_date.strftime('%Y-%m-%d')

    links = get_game_links(date_str)
    for link in links:
        parse_game(ref_foul_data=ref_foul_data,
                   link=link)
        
    # save data in case of crashes
    if scrape_date.day % 15 == 0:
        with open('data.pkl', 'wb') as f:
            pickle.dump(ref_foul_data, f)
        print('data saved on date', scrape_date)
    
    scrape_date += delta

scraping 2023-10-24
scraping 2023-10-25
scraping 2023-10-26
scraping 2023-10-27
scraping 2023-10-28
scraping 2023-10-29
scraping 2023-10-30


ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))